In [2]:
from keras.preprocessing import sequence
import numpy as np
from collections import Counter
import itertools
import time
import pickle
import sets
import keras
from random import shuffle

In [ ]:
documents = []
file = open("data/silva/freshwater/glove_input_freshwater.txt")
for line in file:
    line = line.strip()
    line = line.split("\t")
    documents.append(line)

In [ ]:

dictionary = {}
reverse_dictionary = {}
unlisted = [i for sublist in documents for i in sublist]
freq = Counter(unlisted)
vocab_size = len(freq.keys())

In [ ]:
i = 1
for tup in freq.most_common():
    dictionary[tup[0]] = i
    reverse_dictionary[i] = tup[0]
    i += 1

In [ ]:
freq.most_common()

In [ ]:
docs = []
for doc in documents:
    docs.append([dictionary[taxa] for taxa in doc])
np.random.seed(10)
choice = np.random.choice(range(len(docs)), int(3*len(docs)/4)) 
docs_train = np.array(docs)[choice]
print(len(docs_train))

In [ ]:
sampling_table = sequence.make_sampling_table(vocab_size)

In [ ]:

epochs = 1000000

In [ ]:
print(len(docs_train))

In [ ]:
increment = 10
total_docs = 50
def subsetDoc(i):
    print("subsetting document")
    start = 0 + (increment *i)
    end = increment +  (increment * i)
    docs_small = docs_train[start:end]
    return(docs_small)

In [ ]:
def getCouples(chunk):
    import itertools
    import pickle
    couples = []
    labels = []
    start = 0 + (increment *chunk)
    end = increment +  (increment * chunk)
    docs_small = docs_train[start:end]
    for doc in docs_small:
        pairs = itertools.combinations(doc, r=2)
        for i,j in pairs:

            couples.append([i,j])
            labels.append(1)
    couple_file = open('contexts/couples_' + str(chunk) + '.obj', 'wb')
    label_file = open('contexts/labels_' + str(chunk) + '.obj', 'wb')
    pickle.dump(couples, couple_file)
    pickle.dump(labels, label_file)
    couple_file.close()
    label_file.close()
    
        


In [ ]:
dview = c[:]
dview.push({'docs_train': docs_train})
dview.push({'increment': increment})
#dview.push({'sampling_table': sampling_table})
start_time = time.time()
dview.map_sync(getCouples, range(int(50/10)))
end_time = time.time()
print(end_time - start_time)

In [ ]:
#I don't understand why, but there are a number of samples that are just taxa 18568 and NO other taxa. Not sure why this is
#but pretty sure it's not meaningful data
docs_train_new = []
delete = []
for i in range(len(docs_train)):
    doc = docs_train[i]
    #18568 = number of taxa in full dataset
    #6581 = number of taxa in 5perc dataset
    if doc == [6581]:
        print(i)
        delete.append(i)
docs_train_new = np.delete(docs_train, delete)
print(len(docs_train_new))

In [ ]:
#Make hashtable
ntaxa = len(dictionary.keys())
taxa_dict = []
for i in range(ntaxa):
    taxa_dict.append(set())

sample = 0
for doc in docs_train_new:  
    for taxa in doc:
        taxa_dict[taxa].add(sample)
    sample += 1

del taxa_dict[0]

In [ ]:
print(len(taxa_dict))
#18567

In [ ]:
def getNegEx(id_interest):
    from random import shuffle
    neg_hits = 0
    neg_hit_taxa = []
    shuffle(ids)
    counter = 0
    numNegs = 10
    while neg_hits < numNegs and counter < len(ids):
        numSamplesShared = len(taxa_dict[id_interest].intersection(taxa_dict[ids[counter]]))
        if numSamplesShared == 0:
            neg_hits += 1
            neg_hit_taxa.append(ids[counter])
        counter += 1
    return([id_interest, neg_hit_taxa])

In [ ]:
c = p.Client()
dview = c[:]

ntaxa = len(taxa_dict)
ids = [i for i in range(ntaxa)]
dview.push({'taxa_dict': taxa_dict})
dview.push({'ids': ids})
neg_hits = dview.map_sync(getNegEx, range(10))

In [ ]:
couples_neg = []
labels_neg = []
for block in neg_hits:
    for neg_partner in block[1]:
        couples_neg.append([block[0], neg_partner])
        labels_neg.append(0)

In [ ]:
couples_neg

In [ ]:


neg_couples = []
labels = []
for taxa_id in range(ntaxa):
    neg_ex = getNegEx(taxa_id, numNegs = 2000)
    for neg_taxa_id in neg_ex:
        neg_couples.append([taxa_id, neg_taxa_id])
        labels.append(0)

In [ ]:
neg_couples_file = open('neg_couples.obj', 'wb')
neg_labels_file = open('neg_labels.obj', 'wb')

pickle.dump(neg_couples, neg_couples_file)
pickle.dump(labels, neg_labels_file)
neg_couples_file.close()
neg_labels_file.close()

In [ ]:
neg_labels = labels

In [3]:
from keras.layers import Input, Dense, Embedding, Reshape, Dot
from keras.models import Model 
vector_dim = 500

input_target = Input((1,))
input_context = Input((1,))

embedding = Embedding(vocab_size, vector_dim, input_length=1, name = "embedding")
target = embedding(input_target)
target = Reshape((vector_dim, 1))(target)

context = embedding(input_context)
context = Reshape((vector_dim, 1))(context)

similarity = Dot(axes = 0, normalize = True)([target, context]) # cosine similarity (end up with scalar)

dot_product = Dot(axes = 1, normalize = False)([target, context]) #Component-wise multiply
dot_product = Reshape((1,))(dot_product)

output = Dense(1, activation='sigmoid')(dot_product)

model = Model(inputs=[input_target, input_context], outputs = output)
model.compile(loss = "binary_crossentropy", optimizer = 'rmsprop')


validation_model = Model(inputs=[input_target, input_context], outputs = similarity)


NameError: name 'vocab_size' is not defined

In [ ]:
class SimilarityCallback:
    def run_sim(self):
        for i in range(valid_size):
            valid_word = reverse_dictionary[valid_examples[i]]
            top_k = 8  # number of nearest neighbors
            sim = self._get_sim(valid_examples[i])
            nearest = (-sim).argsort()[1:top_k + 1]
            log_str = 'Nearest to %s:' % valid_word
            for k in range(top_k):
                close_word = reverse_dictionary[nearest[k]]
                log_str = '%s %s,' % (log_str, close_word)
            print(log_str)

    @staticmethod
    def _get_sim(valid_word_idx):
        sim = np.zeros((vocab_size,))
        in_arr1 = np.zeros((1,))
        in_arr2 = np.zeros((1,))
        for i in range(vocab_size):
            in_arr1[0,] = valid_word_idx
            in_arr2[0,] = i
            out = validation_model.predict_on_batch([in_arr1, in_arr2])
            sim[i] = out
        return sim
sim_cb = SimilarityCallback()

In [ ]:
couples_pos_file = open("contexts/pos/couples_0.obj", "rb")
couples_pos = pickle.load(couples_pos_file)

labels_pos_file = open("contexts/pos/labels_0.obj", "rb")
labels_pos = pickle.load(labels_pos_file)

In [ ]:
couples_neg = []
labels_neg = []
for i in range(100):
    couples_neg_file = open("contexts/neg/couples_" + str(i) + ".obj", "rb")
    tmp = pickle.load(couples_neg_file)
    couples_neg = couples_neg + tmp
    couples_neg_file.close()

    labels_neg_file = open("contexts/neg/labels_" + str(i) + ".obj", "rb")
    tmp = pickle.load(labels_neg_file)
    labels_neg = labels_neg + tmp
    labels_neg_file.close()

In [ ]:
print(len(couples_neg))

In [ ]:
couples_use = couples_pos + couples_neg
labels_use = labels_pos + labels_neg

In [ ]:
word_target, word_context = zip(*couples_use)

In [ ]:
epochs = 5
arr_1 = np.zeros((1,))
arr_2 = np.zeros((1,))
arr_3 = np.zeros((1,))
for cnt in range(epochs):
    idx = np.random.randint(0, len(labels)-1)
    arr_1[0,] = word_target[idx]
    arr_2[0,] = word_context[idx]
    arr_3[0,] = labels_use[idx]
    loss = model.train_on_batch([arr_1, arr_2], arr_3)
    if i % 100 == 0:
        print("Iteration {}, loss={}".format(cnt, loss))
    #if cnt % 10000 == 0:
     #   sim_cb.run_sim()